# Testing InceptionResNetV2 network (variation 1)

In this notebook I use the data explored and processed in notebooks 1-3 to see how well the pretrained InceptionResNetV2 model works as a classifier.

In [2]:
import os
import time
import pickle

import numpy as np

from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Activation, Dropout, Flatten, Dense
from keras import backend as K
from keras import metrics

#TODO implement this function
from cars_utils import time_save_model, plot_metrics

### BASIC  IMAGE, AND BATCH SETUP

In [3]:
# Image dimensions:
IMG_WIDTH, IMG_HEIGHT = 299, 299

WORKING_DIR = os.getcwd()
BASE = os.path.join(WORKING_DIR, 'data', 'cars_dataset', 'keras')

SAVE_DIR = os.path.join(WORKING_DIR, 'data', 'IRv2')

### Settings for full training
# TRAIN_DATA_DIR = os.path.join(BASE, 'train')
# VALIDATION_DATA_DIR = os.path.join(BASE, 'validation')
# NB_CLASSES = 196
# NB_TRAIN_SAMPLES = 11329
# NB_VALIDATION_SAMPLES = 2428
# BATCH_SIZE = 32

### Settings for fast checking
# SAVE_DIR = os.path.join(WORKING_DIR, 'data', 'fast_test')
# TRAIN_DATA_DIR = os.path.join(BASE, 'fast', 'train')
# VALIDATION_DATA_DIR = os.path.join(BASE, 'fast', 'validation')
# NB_CLASSES = 196
# NB_TRAIN_SAMPLES = NB_CLASSES*2
# NB_VALIDATION_SAMPLES = NB_CLASSES
# BATCH_SIZE = 4

### Settings for faster checking
SAVE_DIR = os.path.join(WORKING_DIR, 'data', 'fast_v2_test')
TRAIN_DATA_DIR = os.path.join(BASE, 'fast_v2', 'train')
VALIDATION_DATA_DIR = os.path.join(BASE, 'fast_v2', 'validation')
NB_CLASSES = 6
NB_TRAIN_SAMPLES = NB_CLASSES*2
NB_VALIDATION_SAMPLES = NB_CLASSES
BATCH_SIZE = 4

if K.image_data_format() == 'channels_first':
    INPUT_SHAPE = (3, IMG_WIDTH, IMG_HEIGHT)
else:
    INPUT_SHAPE = (IMG_WIDTH, IMG_HEIGHT, 3)

In [4]:
from keras.applications import InceptionResNetV2

conv_total = InceptionResNetV2(
    weights='imagenet',
    include_top=True,
    input_shape=INPUT_SHAPE
)

In [5]:
conv_total.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 299, 299, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 149, 149, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 149, 149, 32) 96          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 149, 149, 32) 0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_2 (

block17_2_mixed (Concatenate)   (None, 17, 17, 384)  0           activation_81[0][0]              
                                                                 activation_84[0][0]              
__________________________________________________________________________________________________
block17_2_conv (Conv2D)         (None, 17, 17, 1088) 418880      block17_2_mixed[0][0]            
__________________________________________________________________________________________________
block17_2 (Lambda)              (None, 17, 17, 1088) 0           block17_1_ac[0][0]               
                                                                 block17_2_conv[0][0]             
__________________________________________________________________________________________________
block17_2_ac (Activation)       (None, 17, 17, 1088) 0           block17_2[0][0]                  
__________________________________________________________________________________________________
conv2d_86 

In [9]:
conv_base = InceptionResNetV2(
    weights='imagenet',
    include_top=False,
    pooling='avg',
    input_shape=INPUT_SHAPE
)

In [10]:
conv_base.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 299, 299, 3)  0                                            
__________________________________________________________________________________________________
conv2d_407 (Conv2D)             (None, 149, 149, 32) 864         input_3[0][0]                    
__________________________________________________________________________________________________
batch_normalization_407 (BatchN (None, 149, 149, 32) 96          conv2d_407[0][0]                 
__________________________________________________________________________________________________
activation_407 (Activation)     (None, 149, 149, 32) 0           batch_normalization_407[0][0]    
__________________________________________________________________________________________________
conv2d_408

__________________________________________________________________________________________________
batch_normalization_478 (BatchN (None, 35, 35, 64)   192         conv2d_478[0][0]                 
__________________________________________________________________________________________________
activation_473 (Activation)     (None, 35, 35, 32)   0           batch_normalization_473[0][0]    
__________________________________________________________________________________________________
activation_475 (Activation)     (None, 35, 35, 32)   0           batch_normalization_475[0][0]    
__________________________________________________________________________________________________
activation_478 (Activation)     (None, 35, 35, 64)   0           batch_normalization_478[0][0]    
__________________________________________________________________________________________________
block35_10_mixed (Concatenate)  (None, 35, 35, 128)  0           activation_473[0][0]             
          

In [ ]:
model = Sequential()
model.add(conv_base)
model.add(Dense(NB_CLASSES, activation='softmax'))

conv_base.trainable = True
print('This is the number of trainable weights '
      'before freezing the conv base:', len(model.trainable_weights))
conv_base.trainable = False
print('This is the number of trainable weights '
      'after freezing the conv base:', len(model.trainable_weights))

In [ ]:
model.summary()

In [ ]:
train_datagen = ImageDataGenerator(rescale=1/255,
                                   rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode='nearest')

test_datagen = ImageDataGenerator(rescale=1/255)

In [ ]:
train_generator = train_datagen.flow_from_directory(
    TRAIN_DATA_DIR,
    target_size=(IMG_WIDTH, IMG_HEIGHT),
    batch_size=BATCH_SIZE,
    class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
    VALIDATION_DATA_DIR,
    target_size=(IMG_WIDTH, IMG_HEIGHT),
    batch_size=BATCH_SIZE,
    class_mode='categorical')

Define custom metrics to measure

In [ ]:
def top_3_acc(y_true, y_pred):
    return metrics.top_k_categorical_accuracy(y_true, y_pred, k=3)

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=[metrics.categorical_accuracy, top_3_acc])

In [ ]:
EPOCHS = 100
start_time = time.time()
summary = model.fit_generator(
    train_generator,
    steps_per_epoch=NB_TRAIN_SAMPLES // BATCH_SIZE,
    epochs=EPOCHS,
    validation_data=validation_generator,
    validation_steps=NB_VALIDATION_SAMPLES // BATCH_SIZE)
end_time = time.time()

In [ ]:
time_save_model(model,
                summary.history,
                start_time, end_time,
                '1_VGG16_0_100',
                SAVE_DIR)

In [ ]:
key_list = ['categorical_accuracy',
            'top_3_acc',
            'loss']
show = plot_metrics(summary.history, key_list)